In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
import os
import time
import sys
import numpy as np
sys.path.append('/home/chaofan/powerknowledge/data')
# sys.path.append('data/')
from read_PLAID_data import read_processed_data,get_feature_name

In [2]:
start_reading_time = time.time()
feature_select = [
    'i_mean', 'i_wave_factor', 'i_pp_rms', 'i_thd', 'pure_thd', 'P', 'Q',
    'P_F', 'i_hp1', 'z_hp1', 'i_hm2', 'z_hm2', 'i_hp2', 'z_hp2', 'i_hm3',
    'z_hm3', 'i_hp3', 'z_hp3', 'i_hm4', 'z_hm4', 'i_hp4', 'z_hp4', 'i_hm5',
    'z_hm5', 'i_hp5', 'z_hp5', 'i_hm6', 'z_hm6', 'i_hp6', 'z_hp6', 'i_hm7',
    'z_hm7', 'i_hp7', 'z_hp7'
]
selected_label = [
    'Air Conditioner', 'Blender', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_process2.1/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_process2.1/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_process2.1/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 11.385s


In [9]:
best_score = 0.0
for n_estimator in range(10, 201, 10):
    estimator = GradientBoostingClassifier(n_estimators=n_estimator,
                                           learning_rate=0.1,
                                           min_samples_split=10,
                                           min_samples_leaf=5,
                                           max_depth=8,
                                           max_features='sqrt',
                                           random_state=10)
    estimator.fit(x_mh_train, y_mh_train)
    score = estimator.score(x_mh_validation, y_mh_validation)
    if score > best_score:
        best_score = score
        best_n_estimator = n_estimator
gbc0 = GradientBoostingClassifier(n_estimators=best_n_estimator,
                                  learning_rate=0.1,
                                  min_samples_split=10,
                                  min_samples_leaf=5,
                                  max_depth=8,
                                  max_features='sqrt',
                                  random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_n_estimator: %02d' % best_n_estimator)


train best score: 0.858537
Accuracy : 0.8129
AUC Score (train): 1.000000
AUC Score (test): 0.984286
best_n_estimator: 130


In [10]:
best_score = 0.0
for md in range(3,15,1):
    for mss in range(2,20,2):
        estimator = GradientBoostingClassifier(n_estimators=130,
                                            learning_rate=0.1,
                                            min_samples_split=mss,
                                            min_samples_leaf=5,
                                            max_depth=md,
                                            max_features='sqrt',
                                            random_state=10)
        estimator.fit(x_mh_train, y_mh_train)
        score = estimator.score(x_mh_validation, y_mh_validation)
        if score > best_score:
            best_score = score
            best_md = md
            best_mss = mss
gbc0 = GradientBoostingClassifier(n_estimators=130,
                                  learning_rate=0.1,
                                  min_samples_split=best_mss,
                                  min_samples_leaf=5,
                                  max_depth=best_md,
                                  max_features='sqrt',
                                  random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_md: %02d, best_mss: %02d' % (best_md,best_mss))

train best score: 0.892683
Accuracy : 0.806
AUC Score (train): 1.000000
AUC Score (test): 0.985178
best_md: 08, best_mss: 16


In [11]:
best_score = 0.0
for msl in range(3,15,1):
    for mss in range(5,51,5):
        estimator = GradientBoostingClassifier(n_estimators=130,
                                            learning_rate=0.1,
                                            min_samples_split=mss,
                                            min_samples_leaf=msl,
                                            max_depth=8,
                                            max_features='sqrt',
                                            random_state=10)
        estimator.fit(x_mh_train, y_mh_train)
        score = estimator.score(x_mh_validation, y_mh_validation)
        if score > best_score:
            best_score = score
            best_msl = msl
            best_mss = mss
gbc0 = GradientBoostingClassifier(n_estimators=130,
                                  learning_rate=0.1,
                                  min_samples_split=best_mss,
                                  min_samples_leaf=best_msl,
                                  max_depth=8,
                                  max_features='sqrt',
                                  random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_msl: %02d, best_mss: %02d' % (best_msl,best_mss))

train best score: 0.887805
Accuracy : 0.8083
AUC Score (train): 1.000000
AUC Score (test): 0.980305
best_msl: 06, best_mss: 15


In [16]:
gbc0 = GradientBoostingClassifier(n_estimators=130,
                                  learning_rate=0.1,
                                  min_samples_split=16,
                                  min_samples_leaf=5,
                                  max_depth=8,
                                  random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))

Accuracy : 0.8083
AUC Score (train): 0.998997
AUC Score (test): 0.981976


In [2]:
# test zengj
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')
selected_label = [
    'Air Conditioner', 'Blender', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 5.120s


In [3]:
gbc0 = GradientBoostingClassifier(n_estimators=140,
                                  learning_rate=0.1,
                                  min_samples_split=16,
                                  min_samples_leaf=5,
                                  max_depth=8,
                                  random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))

Accuracy : 0.7991
AUC Score (train): 0.998997
AUC Score (test): 0.971728


In [4]:
# HIOKI
from read_HIOKI_data import read_processed_data
selected_label=['Fan','Hair Dryer','Kettle']
test_x,test_y=read_processed_data('type',selected_label=selected_label,direaction=1,each_lenth=1)
test_y=test_y.reshape(-1,1)
pred_y = gbc0.predict(test_x).reshape(-1,1)
plot_y=np.concatenate((test_y,pred_y),axis=1)

In [2]:
# test zengj dianzi
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')
selected_label = [
   'Laptop','Compact Fluorescent Lamp'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

gbc1 = GradientBoostingClassifier(n_estimators=140,
                                  learning_rate=0.1,
                                  min_samples_split=16,
                                  min_samples_leaf=5,
                                  max_depth=8,
                                  random_state=10)
gbc1.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc1.predict(x_mh_trainval)
y_trainval_predprob = gbc1.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc1.predict(x_mh_test)
y_test_predprob = gbc1.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))

finished loading data, cost 4.830s
Accuracy : 0.8582


ValueError: y should be a 1d array, got an array of shape (312, 2) instead.

In [3]:
# HIOKI
from read_HIOKI_data import read_processed_data
selected_label=['Incandescent Lamp','Table Lamp']
test_x,test_y=read_processed_data('type',selected_label=selected_label,direaction=1,each_lenth=1)
test_y=test_y.reshape(-1,1)
pred_y = gbc1.predict(test_x).reshape(-1,1)
plot_y=np.concatenate((test_y,pred_y),axis=1)

In [2]:
# 小功率
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')
selected_label = [
    'Fan', 'Fridge', 'Hair Iron',
    'Incandescent Light Bulb'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

gbc0 = GradientBoostingClassifier(n_estimators=140,
                                  learning_rate=0.1,
                                  min_samples_split=16,
                                  min_samples_leaf=5,
                                  max_depth=8,
                                  random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))

finished loading data, cost 4.963s
Accuracy : 0.94
AUC Score (train): 1.000000
AUC Score (test): 0.993244


In [3]:
from read_HIOKI_data import read_processed_data
selected_label=['Fan','Incandescent Lamp']
test_x,test_y=read_processed_data('type',selected_label=selected_label,direaction=1,each_lenth=1)
test_y=test_y.reshape(-1,1)
pred_y = gbc0.predict(test_x).reshape(-1,1)
plot_y=np.concatenate((test_y,pred_y),axis=1)

In [2]:
# 大功率
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')
selected_label = [
    'Air Conditioner', 'Blender', 'Coffee maker',
    'Hairdryer', 'Heater', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

gbc0 = GradientBoostingClassifier(n_estimators=140,
                                  learning_rate=0.1,
                                  min_samples_split=16,
                                  min_samples_leaf=5,
                                  max_depth=8,
                                  random_state=10)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))

finished loading data, cost 5.051s
Accuracy : 0.7809
AUC Score (train): 0.998472
AUC Score (test): 0.966984


In [3]:
from read_HIOKI_data import read_processed_data
selected_label=['Hair Dryer','Kettle']
test_x,test_y=read_processed_data('type',selected_label=selected_label,direaction=1,each_lenth=1)
test_y=test_y.reshape(-1,1)
pred_y = gbc0.predict(test_x).reshape(-1,1)
plot_y=np.concatenate((test_y,pred_y),axis=1)